In [4]:
# import packages

import pandas as pd
import numpy as np
from pathlib import Path
from functions import (load_month, load_month_sa, load_month_pen, mapd_clean_merge)

# Settings
monthlist = [f"{m:02d}" for m in range(1, 3)]  # "01", "02"
y = 2014

In [5]:
# Load and combine monthly data
plan_data = pd.concat([load_month(m, y) for m in monthlist], ignore_index=True)

# Sort data
plan_data = plan_data.sort_values(['contractid', 'planid', 'state', 'county', 'month'])

# Fill fips by state/county groups
plan_data['fips'] = plan_data.groupby(['state', 'county'])['fips'].transform(
    lambda x: x.ffill().bfill()
)

# Fill plan-level attributes
plan_cols = ['plan_type', 'partd', 'snp', 'eghp', 'plan_name']
for col in plan_cols:
    plan_data[col] = plan_data.groupby(['contractid', 'planid'])[col].transform(
        lambda x: x.ffill().bfill()
    )

# Fill contract-level attributes
contract_cols = ['org_type', 'org_name', 'org_marketing_name', 'parent_org']
for col in contract_cols:
    plan_data[col] = plan_data.groupby(['contractid'])[col].transform(
        lambda x: x.ffill().bfill()
    )

# Sort for aggregation
plan_data = plan_data.sort_values(['contractid', 'planid', 'fips', 'year', 'month'])

# Aggregate to yearly level
def agg_plan_year(group):
    nonmiss = group['enrollment'].notna()
    n = nonmiss.sum()
    enroll_vals = group.loc[nonmiss, 'enrollment']
    
    return pd.Series({
        'n_nonmiss': n,
        'avg_enrollment': enroll_vals.mean() if n > 0 else np.nan,
        'sd_enrollment': enroll_vals.std() if n > 1 else np.nan,
        'min_enrollment': enroll_vals.min() if n > 0 else np.nan,
        'max_enrollment': enroll_vals.max() if n > 0 else np.nan,
        'first_enrollment': enroll_vals.iloc[0] if n > 0 else np.nan,
        'last_enrollment': enroll_vals.iloc[-1] if n > 0 else np.nan,
        'state': group['state'].iloc[-1],
        'county': group['county'].iloc[-1],
        'org_type': group['org_type'].iloc[-1],
        'plan_type': group['plan_type'].iloc[-1],
        'partd': group['partd'].iloc[-1],
        'snp': group['snp'].iloc[-1],
        'eghp': group['eghp'].iloc[-1],
        'org_name': group['org_name'].iloc[-1],
        'org_marketing_name': group['org_marketing_name'].iloc[-1],
        'plan_name': group['plan_name'].iloc[-1],
        'parent_org': group['parent_org'].iloc[-1],
        'contract_date': group['contract_date'].iloc[-1]
    })

plan_data_2014 = plan_data.groupby(['contractid', 'planid', 'fips', 'year']).apply(agg_plan_year).reset_index()
print(f"Plan data shape: {plan_data_2014.shape}")

/tmp/ipykernel_1044209/811496249.py:16: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  lambda x: x.ffill().bfill()
/tmp/ipykernel_1044209/811496249.py:23: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  lambda x: x.ffill().bfill()
/tmp/ipykernel_1044209/811496249.py:57: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping colum

Plan data shape: (2404587, 23)


In [6]:
## Plan enrollment/contract data

# Load and combine monthly data
plan_data = pd.concat([load_month(m, y) for m in monthlist], ignore_index=True)

# Sort data
plan_data = plan_data.sort_values(['contractid', 'planid', 'state', 'county', 'month'])

# Fill fips by state/county groups
plan_data['fips'] = plan_data.groupby(['state', 'county'])['fips'].transform(
    lambda x: x.ffill().bfill()
)

# Fill plan-level attributes
plan_cols = ['plan_type', 'partd', 'snp', 'eghp', 'plan_name']
for col in plan_cols:
    plan_data[col] = plan_data.groupby(['contractid', 'planid'])[col].transform(
        lambda x: x.ffill().bfill()
    )

# Fill contract-level attributes
contract_cols = ['org_type', 'org_name', 'org_marketing_name', 'parent_org']
for col in contract_cols:
    plan_data[col] = plan_data.groupby(['contractid'])[col].transform(
        lambda x: x.ffill().bfill()
    )

# Sort for aggregation
plan_data = plan_data.sort_values(['contractid', 'planid', 'fips', 'year', 'month'])

# Aggregate to yearly level
def agg_plan_year(group):
    nonmiss = group['enrollment'].notna()
    n = nonmiss.sum()
    enroll_vals = group.loc[nonmiss, 'enrollment']
    
    return pd.Series({
        'n_nonmiss': n,
        'avg_enrollment': enroll_vals.mean() if n > 0 else np.nan,
        'sd_enrollment': enroll_vals.std() if n > 1 else np.nan,
        'min_enrollment': enroll_vals.min() if n > 0 else np.nan,
        'max_enrollment': enroll_vals.max() if n > 0 else np.nan,
        'first_enrollment': enroll_vals.iloc[0] if n > 0 else np.nan,
        'last_enrollment': enroll_vals.iloc[-1] if n > 0 else np.nan,
        'state': group['state'].iloc[-1],
        'county': group['county'].iloc[-1],
        'org_type': group['org_type'].iloc[-1],
        'plan_type': group['plan_type'].iloc[-1],
        'partd': group['partd'].iloc[-1],
        'snp': group['snp'].iloc[-1],
        'eghp': group['eghp'].iloc[-1],
        'org_name': group['org_name'].iloc[-1],
        'org_marketing_name': group['org_marketing_name'].iloc[-1],
        'plan_name': group['plan_name'].iloc[-1],
        'parent_org': group['parent_org'].iloc[-1],
        'contract_date': group['contract_date'].iloc[-1]
    })

plan_data_2014 = plan_data.groupby(['contractid', 'planid', 'fips', 'year']).apply(agg_plan_year).reset_index()
print(f"Plan data shape: {plan_data_2014.shape}")

/tmp/ipykernel_1044209/1688086670.py:18: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  lambda x: x.ffill().bfill()
/tmp/ipykernel_1044209/1688086670.py:25: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  lambda x: x.ffill().bfill()


KeyboardInterrupt: 

In [7]:
## Service area data

# Load and combine monthly service area data
service_year = pd.concat([load_month_sa(m, y) for m in monthlist], ignore_index=True)

# Sort for stable fills
service_year = service_year.sort_values(['contractid', 'fips', 'state', 'county', 'month'])

# Fill fips by state/county groups
service_year['fips'] = service_year.groupby(['state', 'county'])['fips'].transform(
    lambda x: x.ffill().bfill()
)

# Fill contract-level attributes
contract_cols_sa = ['plan_type', 'partial', 'eghp', 'org_type', 'org_name']
for col in contract_cols_sa:
    service_year[col] = service_year.groupby(['contractid'])[col].transform(
        lambda x: x.ffill().bfill()
    )

# Collapse to yearly: one row per contract x county (fips) x year
service_year = service_year.sort_values(['contractid', 'fips', 'year', 'month'])

service_data_2014 = service_year.groupby(['contractid', 'fips', 'year']).agg({
    'state': 'last',
    'county': 'last',
    'org_name': 'last',
    'org_type': 'last',
    'plan_type': 'last',
    'partial': 'last',
    'eghp': 'last',
    'ssa': 'last',
    'notes': 'last'
}).reset_index()

print(f"Service area data shape: {service_data_2014.shape}")

/tmp/ipykernel_1044209/4112319176.py:18: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  lambda x: x.ffill().bfill()
/tmp/ipykernel_1044209/4112319176.py:18: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  lambda x: x.ffill().bfill()


Service area data shape: (397035, 12)


In [9]:
## Plan characteristics (premium) data

# MA landscape data (A to M)
ma_path_a = "../ma-data/ma/landscape/Extracted Data/2014LandscapeSource file MA_AtoM 05292014.csv"
ma_col_names = ["state", "county", "org_name", "plan_name", "plan_type", "premium", "partd_deductible",
                "drug_type", "gap_coverage", "drug_type_detail", "contractid",
                "planid", "segmentid", "moop", "star_rating"]

ma_data_a = pd.read_csv(ma_path_a, skiprows=6, names=ma_col_names, dtype=str, encoding = "latin-1")
ma_data_a['premium'] = ma_data_a['premium'].str.replace('-', '0')
ma_data_a['partd_deductible'] = ma_data_a['partd_deductible'].str.replace('-', '0')
ma_data_a['premium'] = pd.to_numeric(ma_data_a['premium'].str.replace(r'[^\d.]', '', regex=True), errors='coerce')
ma_data_a['partd_deductible'] = pd.to_numeric(ma_data_a['partd_deductible'].str.replace(r'[^\d.]', '', regex=True), errors='coerce')
ma_data_a['planid'] = pd.to_numeric(ma_data_a['planid'], errors='coerce')
ma_data_a['segmentid'] = pd.to_numeric(ma_data_a['segmentid'], errors='coerce')

# MA landscape data (N to W)
ma_path_b = "../ma-data/ma/landscape/Extracted Data/2014LandscapeSource file MA_NtoW 05292014.csv"
ma_data_b = pd.read_csv(ma_path_b, skiprows=6, names=ma_col_names, dtype=str, encoding = "latin-1")
ma_data_b['premium'] = ma_data_b['premium'].str.replace('-', '0')
ma_data_b['partd_deductible'] = ma_data_b['partd_deductible'].str.replace('-', '0')
ma_data_b['premium'] = pd.to_numeric(ma_data_b['premium'].str.replace(r'[^\d.]', '', regex=True), errors='coerce')
ma_data_b['partd_deductible'] = pd.to_numeric(ma_data_b['partd_deductible'].str.replace(r'[^\d.]', '', regex=True), errors='coerce')
ma_data_b['planid'] = pd.to_numeric(ma_data_b['planid'], errors='coerce')
ma_data_b['segmentid'] = pd.to_numeric(ma_data_b['segmentid'], errors='coerce')

ma_data = pd.concat([ma_data_a, ma_data_b], ignore_index=True)

# MA-PD landscape data
mapd_path = "../ma-data/ma/landscape/Extracted Data/PartCD/2014/Medicare Part D 2014 Plan Report 05292014.xls"
mapd_col_names = ["state", "county", "org_name", "plan_name", "contractid", "planid", "segmentid",
                  "org_type", "plan_type", "snp", "snp_type", "benefit_type", "below_benchmark",
                  "national_pdp", "premium_partc",
                  "premium_partd_basic", "premium_partd_supp", "premium_partd_total",
                  "partd_assist_full", "partd_assist_75", "partd_assist_50", "partd_assist_25",
                  "partd_deductible", "deductible_exclusions", "increase_coverage_limit",
                  "gap_coverage", "gap_coverage_type"]

mapd_data_a = pd.read_excel(mapd_path, sheet_name="Alabama to Montana", skiprows=4, 
                            nrows=15854, names=mapd_col_names)
mapd_data_b = pd.read_excel(mapd_path, sheet_name="Nebraska to Wyoming", skiprows=4,
                            nrows=20300, names=mapd_col_names)
mapd_data = pd.concat([mapd_data_a, mapd_data_b], ignore_index=True)

# Clean and merge
landscape_2014 = mapd_clean_merge(ma_data, mapd_data, y)
print(f"Landscape data shape: {landscape_2014.shape}")

Landscape data shape: (45313, 11)


In [10]:
## Penetration data

# Load and combine monthly penetration data
ma_penetration = pd.concat([load_month_pen(m, y) for m in monthlist], ignore_index=True)

# Sort and fill fips
ma_penetration = ma_penetration.sort_values(['state', 'county', 'month'])
ma_penetration['fips'] = ma_penetration.groupby(['state', 'county'])['fips'].transform(
    lambda x: x.ffill().bfill()
)

# Collapse to yearly
def agg_penetration(group):
    n_elig = group['eligibles'].notna().sum()
    n_enrol = group['enrolled'].notna().sum()
    elig_vals = group['eligibles'].dropna()
    enrol_vals = group['enrolled'].dropna()
    
    return pd.Series({
        'n_elig': n_elig,
        'n_enrol': n_enrol,
        'avg_eligibles': elig_vals.mean() if n_elig > 0 else np.nan,
        'sd_eligibles': elig_vals.std() if n_elig > 1 else np.nan,
        'min_eligibles': elig_vals.min() if n_elig > 0 else np.nan,
        'max_eligibles': elig_vals.max() if n_elig > 0 else np.nan,
        'first_eligibles': elig_vals.iloc[0] if n_elig > 0 else np.nan,
        'last_eligibles': elig_vals.iloc[-1] if n_elig > 0 else np.nan,
        'avg_enrolled': enrol_vals.mean() if n_enrol > 0 else np.nan,
        'sd_enrolled': enrol_vals.std() if n_enrol > 1 else np.nan,
        'min_enrolled': enrol_vals.min() if n_enrol > 0 else np.nan,
        'max_enrolled': enrol_vals.max() if n_enrol > 0 else np.nan,
        'first_enrolled': enrol_vals.iloc[0] if n_enrol > 0 else np.nan,
        'last_enrolled': enrol_vals.iloc[-1] if n_enrol > 0 else np.nan,
        'ssa': group['ssa'].iloc[-1]
    })

pen_2014 = ma_penetration.groupby(['fips', 'state', 'county', 'year']).apply(agg_penetration).reset_index()
print(f"Penetration data shape: {pen_2014.shape}")

Penetration data shape: (3224, 19)


/tmp/ipykernel_1044209/1772146158.py:37: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  pen_2014 = ma_penetration.groupby(['fips', 'state', 'county', 'year']).apply(agg_penetration).reset_index()


In [12]:
## Rebate data

# Part C rebate data
ma_path_a = "../ma-data/ma/cms-payment/2014/2014PartCPlan Level.xlsx"
risk_rebate_a = pd.read_excel(ma_path_a, skiprows=3, nrows=2824,
                               names=["contractid", "planid", "contract_name", "plan_type",
                                      "riskscore_partc", "payment_partc", "rebate_partc"])

# Clean Part C data
for col in ['riskscore_partc', 'payment_partc', 'rebate_partc']:
    risk_rebate_a[col] = pd.to_numeric(risk_rebate_a[col].astype(str).str.replace(r'[^\d.-]', '', regex=True), errors='coerce')
risk_rebate_a['planid'] = pd.to_numeric(risk_rebate_a['planid'], errors='coerce')
risk_rebate_a['year'] = 2014

# Part D rebate data
ma_path_b = "../ma-data/ma/cms-payment/2014/2014PartDPlans.xlsx"
risk_rebate_b = pd.read_excel(ma_path_b, skiprows=3, nrows=3898,
                               names=["contractid", "planid", "contract_name", "plan_type",
                                      "directsubsidy_partd", "riskscore_partd", "reinsurance_partd",
                                      "costsharing_partd"])

# Clean Part D data
for col in ['directsubsidy_partd', 'reinsurance_partd', 'costsharing_partd']:
    risk_rebate_b[col] = pd.to_numeric(risk_rebate_b[col].astype(str).str.replace(r'[^\d.-]', '', regex=True), errors='coerce')
risk_rebate_b['payment_partd'] = (risk_rebate_b['directsubsidy_partd'] + 
                                   risk_rebate_b['reinsurance_partd'] + 
                                   risk_rebate_b['costsharing_partd'])
risk_rebate_b['planid'] = pd.to_numeric(risk_rebate_b['planid'], errors='coerce')

# Select relevant columns for Part D
risk_rebate_b = risk_rebate_b[['contractid', 'planid', 'payment_partd',
                               'directsubsidy_partd', 'reinsurance_partd', 'costsharing_partd',
                               'riskscore_partd']]

# Merge Part C and Part D
rebate_2014 = risk_rebate_a.merge(risk_rebate_b, on=['contractid', 'planid'], how='left')
print(f"Rebate data shape: {rebate_2014.shape}")

Rebate data shape: (2824, 13)


In [14]:
## FFS costs data

# FFS costs data
ffs_path = "../ma-data/ffs-costs/Extracted Data/Aged Only/aged14.csv"
ffs_col_names = ["ssa", "state", "county_name", "parta_enroll",
                 "parta_reimb", "parta_percap", "parta_reimb_unadj",
                 "parta_percap_unadj", "parta_ime", "parta_dsh",
                 "parta_gme", "partb_enroll",
                 "partb_reimb", "partb_percap",
                 "mean_risk"]

ffs_data = pd.read_csv(ffs_path, skiprows=2, names=ffs_col_names, na_values=['*', '.'],
                       usecols=range(15))

# Select and clean
ffscosts_2014 = ffs_data[['ssa', 'state', 'county_name', 'parta_enroll', 'parta_reimb',
                          'partb_enroll', 'partb_reimb', 'mean_risk']].copy()
ffscosts_2014['year'] = 2014
ffscosts_2014['ssa'] = pd.to_numeric(ffscosts_2014['ssa'], errors='coerce')

for col in ['parta_enroll', 'parta_reimb', 'partb_enroll', 'partb_reimb', 'mean_risk']:
    ffscosts_2014[col] = pd.to_numeric(ffscosts_2014[col].astype(str).str.replace(r'[^\d.-]', '', regex=True), errors='coerce')

print(f"FFS costs data shape: {ffscosts_2014.shape}")

# compute avg_ffs
ae = ffscosts_2014["parta_enroll"].fillna(0)
be = ffscosts_2014["partb_enroll"].fillna(0)
ar = ffscosts_2014["parta_reimb"]
br = ffscosts_2014["partb_reimb"]

ffscosts_2014["avg_ffscost"] = np.where(
    (ae == 0) & (be == 0), 0,
    np.where(
        (ae == 0) & (be > 0), br / be,
        np.where(
            (ae > 0) & (be == 0), ar / ae,
            (ar / ae) + (br / be)
        )
    )
)

FFS costs data shape: (3225, 9)


In [18]:
## Merge data

# Start with plan data, inner join with service area
ma_2014 = plan_data_2014.merge(
    service_data_2014[['contractid', 'fips']],
    on=['contractid', 'fips'],
    how='inner'
)

# Filter out territories and certain plan types
excluded_states = ['VI', 'PR', 'MP', 'GU', 'AS', '']
ma_2014 = ma_2014[
    (~ma_2014['state'].isin(excluded_states)) &
    (ma_2014['snp'] == 'No') &
    ((ma_2014['planid'] < 800) | (ma_2014['planid'] >= 900)) &
    (ma_2014['planid'].notna()) &
    (ma_2014['fips'].notna())
]

# Prepare penetration data for join
pen_2014_join = pen_2014.copy()
pen_2014_join = pen_2014_join.rename(columns={'state': 'state_long', 'county': 'county_long'})
pen_2014_join['state_long'] = pen_2014_join['state_long'].str.lower()

# Keep only unique fips entries
pen_2014_join['ncount'] = pen_2014_join.groupby('fips')['fips'].transform('count')
pen_2014_join = pen_2014_join[pen_2014_join['ncount'] == 1].drop(columns=['ncount'])

# Join penetration data
ma_2014 = ma_2014.merge(pen_2014_join, on='fips', how='left', suffixes=('', '_pen'))

# Create state name lookup
state_2014 = ma_2014.groupby('state').apply(
    lambda g: g.loc[g['state_long'].notna(), 'state_long'].iloc[-1] if g['state_long'].notna().any() else None
).reset_index()
state_2014.columns = ['state', 'state_name']

# Join state names
full_2014 = ma_2014.merge(state_2014, on='state', how='left')

# Prepare landscape data for join
landscape_2014_join = landscape_2014.copy()
landscape_2014_join['state'] = landscape_2014_join['state'].str.lower()

# Join landscape data
full_2014 = full_2014.merge(
    landscape_2014_join,
    left_on=['contractid', 'planid', 'state_name', 'county'],
    right_on=['contractid', 'planid', 'state', 'county'],
    how='left',
    suffixes=('', '_land')
)

# Join rebate data (exclude contract_name and plan_type from rebate)
rebate_2014_join = rebate_2014.drop(columns=['contract_name', 'plan_type'], errors='ignore')
full_2014 = full_2014.merge(rebate_2014_join, on=['contractid', 'planid'], how='left', suffixes=('', '_reb'))

# Calculate basic_premium and bid
def calc_basic_premium(row):
    if row.get('rebate_partc', 0) > 0:
        return 0
    elif row.get('partd') == 'No' and pd.notna(row.get('premium')) and pd.isna(row.get('premium_partc')):
        return row.get('premium')
    else:
        return row.get('premium_partc')

def calc_bid(row):
    rebate = row.get('rebate_partc', 0) or 0
    basic_premium = row.get('basic_premium', 0) or 0
    payment_partc = row.get('payment_partc')
    riskscore_partc = row.get('riskscore_partc')
    
    if pd.isna(payment_partc) or pd.isna(riskscore_partc) or riskscore_partc == 0:
        return np.nan
    elif rebate == 0 and basic_premium > 0:
        return (payment_partc + basic_premium) / riskscore_partc
    elif rebate > 0 or basic_premium == 0:
        return payment_partc / riskscore_partc
    else:
        return np.nan

# Ensure SSA types match
full_2014['ssa'] = pd.to_numeric(full_2014['ssa'], errors='coerce')
ffscosts_2014['ssa'] = pd.to_numeric(ffscosts_2014['ssa'], errors='coerce')

# Merge FFS costs
full_2014 = full_2014.merge(
    ffscosts_2014[['ssa', 'avg_ffscost']],
    on='ssa',
    how='left'
)

print("After FFS merge:", full_2014.shape)


full_2014['basic_premium'] = full_2014.apply(calc_basic_premium, axis=1)
full_2014['bid'] = full_2014.apply(calc_bid, axis=1)

print(f"Final merged data shape: {full_2014.shape}")

/tmp/ipykernel_1044209/158553173.py:33: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  state_2014 = ma_2014.groupby('state').apply(


After FFS merge: (49892, 60)
Final merged data shape: (49892, 62)


In [19]:
# Save to CSV
output_path = "../data/output/data-2014.csv"
Path(output_path).parent.mkdir(parents=True, exist_ok=True)
full_2014.to_csv(output_path, index=False)
print(f"Data saved to {output_path}")

Data saved to ../data/output/data-2014.csv


In [17]:
print(full_2014.columns)
print(ffscosts_2014.columns)


Index(['contractid', 'planid', 'fips', 'year', 'n_nonmiss', 'avg_enrollment',
       'sd_enrollment', 'min_enrollment', 'max_enrollment', 'first_enrollment',
       'last_enrollment', 'state', 'county', 'org_type', 'plan_type', 'partd',
       'snp', 'eghp', 'org_name', 'org_marketing_name', 'plan_name',
       'parent_org', 'contract_date', 'state_long', 'county_long', 'year_pen',
       'n_elig', 'n_enrol', 'avg_eligibles', 'sd_eligibles', 'min_eligibles',
       'max_eligibles', 'first_eligibles', 'last_eligibles', 'avg_enrolled',
       'sd_enrolled', 'min_enrolled', 'max_enrolled', 'first_enrolled',
       'last_enrolled', 'ssa', 'state_name', 'state_land', 'premium',
       'premium_partc', 'premium_partd_basic', 'premium_partd_supp',
       'premium_partd_total', 'partd_deductible', 'year_land',
       'riskscore_partc', 'payment_partc', 'rebate_partc', 'year_reb',
       'payment_partd', 'directsubsidy_partd', 'reinsurance_partd',
       'costsharing_partd', 'riskscore_partd', 